In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, MaxPool2D
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [2]:
#img_full =[]
#img_vol = []
images = []
def import_maps(path):        
        for index, name in enumerate(os.listdir(path)):
            folder = os.path.join(path, name)
            for file_class in os.listdir(folder):
                im_folder = os.path.join(folder, file_class)
               
                for im in os.listdir(im_folder):
                    img = cv2.imread(os.path.join(im_folder, im))
                    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                    img = cv2.resize(img, (196, 196))
                    if img is not None:
                        img = (img-np.mean(img))/np.std(img)
                        images.append((np.array(img), index)) 
               # mean_vol = np.mean(images)
                #dev_vol = np.std(images)
                #images = (images-mean_vol)/dev_vol
               # print(np.std(images))
                #img_arr = np.reshape(np.array(images), (196,196,150))
               # img_vol.append((img_arr, index))
        return images

In [3]:
image_set = import_maps(r'D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes')

In [4]:
images_all = [i[0] for i in image_set]
images_array = np.array(images_all)
images_array=np.expand_dims(images_array,axis=3)
image_label = [i[1] for i in image_set]
image_label = np.array(image_label)

In [5]:
enc = OneHotEncoder(handle_unknown='ignore')
train_label_enc = enc.fit_transform(image_label.reshape(-1, 1)).toarray()

In [6]:
model = Sequential()
model.add(Conv2D(input_shape=(196,196,1),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
model.add(Flatten(name='flatten'))
model.add(Dense(512, activation='relu', name='fc1'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', name='fc2'))
model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax', name='output'))

model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.Adadelta(0.001),
               metrics=['accuracy'])

model.summary()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                            patience=15, min_lr=0.00001)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 196, 196, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 64)      18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 196, 196, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 98, 98, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 98, 98, 128)       5

In [7]:
history = model.fit(images_array,train_label_enc,validation_split=0.2, batch_size=32, epochs=200, verbose=1,shuffle=True, callbacks = [reduce_lr])

Train on 11880 samples, validate on 2970 samples
Epoch 1/200
11880/11880 [==============================] - 80s 7ms/sample - loss: 1.6658 - accuracy: 0.4291 - val_loss: 2.6660 - val_accuracy: 0.1512
Epoch 2/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 1.1513 - accuracy: 0.5290 - val_loss: 4.3004 - val_accuracy: 0.0010 - accu - ETA: 53s - loss: 1.2292 - accuracy: - ETA: 51s - loss: 1.2292 - accuracy: 0 - ETA: 50s - loss: 1.23
Epoch 3/200
11880/11880 [==============================] - 73s 6ms/sample - loss: 0.9423 - accuracy: 0.6066 - val_loss: 5.1874 - val_accuracy: 3.3670e-04s
Epoch 4/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.7924 - accuracy: 0.6716 - val_loss: 5.2368 - val_accuracy: 0.0013
Epoch 5/200
11880/11880 [==============================] - 73s 6ms/sample - loss: 0.6450 - accuracy: 0.7390 - val_loss: 5.6537 - val_accuracy: 0.0000e+00
Epoch 6/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0

11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0543 - accuracy: 0.9813 - val_loss: 8.5049 - val_accuracy: 0.0024
Epoch 54/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0565 - accuracy: 0.9806 - val_loss: 8.5068 - val_accuracy: 0.0024
Epoch 55/200
11880/11880 [==============================] - 73s 6ms/sample - loss: 0.0544 - accuracy: 0.9812 - val_loss: 8.5046 - val_accuracy: 0.0024
Epoch 56/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0521 - accuracy: 0.9822 - val_loss: 8.4916 - val_accuracy: 0.0024
Epoch 57/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0565 - accuracy: 0.9795 - val_loss: 8.5063 - val_accuracy: 0.0024
Epoch 58/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0533 - accuracy: 0.9827 - val_loss: 8.4854 - val_accuracy: 0.0024
Epoch 59/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0546 - accuracy: 0.981

11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0523 - accuracy: 0.9818 - val_loss: 8.5764 - val_accuracy: 0.0024
Epoch 107/200
11880/11880 [==============================] - 73s 6ms/sample - loss: 0.0483 - accuracy: 0.9836 - val_loss: 8.5604 - val_accuracy: 0.0024
Epoch 108/200
11880/11880 [==============================] - 73s 6ms/sample - loss: 0.0491 - accuracy: 0.9830 - val_loss: 8.5557 - val_accuracy: 0.0024
Epoch 109/200
11880/11880 [==============================] - 73s 6ms/sample - loss: 0.0528 - accuracy: 0.9815 - val_loss: 8.5571 - val_accuracy: 0.0024
Epoch 110/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0502 - accuracy: 0.9823 - val_loss: 8.5670 - val_accuracy: 0.0024
Epoch 111/200
11880/11880 [==============================] - 73s 6ms/sample - loss: 0.0502 - accuracy: 0.9835 - val_loss: 8.5715 - val_accuracy: 0.0024
Epoch 112/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0524 - accuracy:

11880/11880 [==============================] - 73s 6ms/sample - loss: 0.0485 - accuracy: 0.9830 - val_loss: 8.6318 - val_accuracy: 0.0024
Epoch 159/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0494 - accuracy: 0.9821 - val_loss: 8.6383 - val_accuracy: 0.0024
Epoch 160/200
11880/11880 [==============================] - 73s 6ms/sample - loss: 0.0459 - accuracy: 0.9841 - val_loss: 8.6353 - val_accuracy: 0.0024
Epoch 161/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0465 - accuracy: 0.9848 - val_loss: 8.6390 - val_accuracy: 0.0024
Epoch 162/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0469 - accuracy: 0.9848 - val_loss: 8.6366 - val_accuracy: 0.0024
Epoch 163/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0462 - accuracy: 0.9851 - val_loss: 8.6156 - val_accuracy: 0.0024
Epoch 164/200
11880/11880 [==============================] - 72s 6ms/sample - loss: 0.0480 - accuracy: